## Import Dependencies

In [86]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


## Foursquare Credentials and Verson

In [87]:
CLIENT_ID = 'FC5WMF5KAA2YQG5ICBTLLB1IL3POC450QMGGHLNYZMKSUAO5' # your Foursquare ID
CLIENT_SECRET = 'I4XMIHNH0OVB5MVYBAO4BPVRRRD4DWJQKUTUBNXWOD4GUQAI' # your Foursquare Secret
VERSION = '20191008'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FC5WMF5KAA2YQG5ICBTLLB1IL3POC450QMGGHLNYZMKSUAO5
CLIENT_SECRET:I4XMIHNH0OVB5MVYBAO4BPVRRRD4DWJQKUTUBNXWOD4GUQAI


## Import New York data

### Use wget command to import

In [88]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


### Define new variable using 'features' key

In [89]:
neighborhoods_data = newyork_data['features']

### Convert to pandas dataframe

In [90]:
neighborhoods_data = newyork_data['features']

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)
    
 # define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)   

neighborhoods


,Borough,Neighborhood,Latitude,Longitude


In [91]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
    

In [92]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### Use geopy library to get the coordinates 

In [108]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [109]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Use Foursquare API to search for Yoga Studios in New York City

In [110]:
search_query = 'Yoga'
radius = 10000
print(search_query + ' .... OK!')

Yoga .... OK!


In [111]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FC5WMF5KAA2YQG5ICBTLLB1IL3POC450QMGGHLNYZMKSUAO5&client_secret=I4XMIHNH0OVB5MVYBAO4BPVRRRD4DWJQKUTUBNXWOD4GUQAI&ll=40.7127281,-74.0060152&v=20191008&query=Yoga&radius=10000&limit=100'

In [112]:
results = requests.get(url).json()

In [113]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)


In [114]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe1.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Kula Yoga Project,Yoga Studio,28 Warren St,US,New York,United States,Church St,251,"[28 Warren St (Church St), New York, NY 10007,...","[{'label': 'display', 'lat': 40.71434229015808...",40.714342,-74.008094,NaN,10007,NY,4ad79413f964a520610c21e3
1,Modo Yoga NYC,Yoga Studio,434 Avenue of the Americas,US,New York,United States,10th Street,2520,"[434 Avenue of the Americas (10th Street), New...","[{'label': 'display', 'lat': 40.73467444734881...",40.734674,-73.998670,NaN,10011,NY,4f2425cce4b0e12081504088
2,Yoga Vida,Yoga Studio,666 Broadway,US,New York,United States,at Bond St,1831,"[666 Broadway (at Bond St), New York, NY 10012...","[{'label': 'display', 'lat': 40.7270016507027,...",40.727002,-73.995225,NaN,10012,NY,4de56910fa764cf4bec7dd4e
3,Yoga to the People,Yoga Studio,12 Saint Marks Pl,US,New York,United States,btwn 2nd & 3rd Ave.,2279,"[12 Saint Marks Pl (btwn 2nd & 3rd Ave.), New ...","[{'label': 'display', 'lat': 40.72889410039975...",40.728894,-73.989436,NaN,10003,NY,45367ca4f964a520ab3b1fe3
4,Yoga Vida,Yoga Studio,99 University Pl,US,New York,United States,6th Floor,2602,"[99 University Pl (6th Floor), New York, NY 10...","[{'label': 'display', 'lat': 40.73385099517913...",40.733851,-73.992802,NaN,10003,NY,4b58d651f964a520e36d28e3
5,Y7 Yoga,Yoga Studio,E 11th St,US,New York,United States,btwn Broadway & University Pl,2534,"[E 11th St (btwn Broadway & University Pl), Ne...","[{'label': 'display', 'lat': 40.733007, 'lng':...",40.733007,-73.992346,NaN,NaN,NY,5707fa79498e89088b4bf74c
6,SKY TING YOGA,Yoga Studio,381 Broadway,US,New York,United States,NaN,650,"[381 Broadway, New York, NY 10013, United States]","[{'label': 'display', 'lat': 40.71812415369316...",40.718124,-74.003051,NaN,10013,NY,582dbaabd21e031c1a35423d
7,Jivamukti Yoga School NYC,Yoga Studio,841 Broadway Fl 2,US,New York,United States,btwn 13th & 14th St,2709,"[841 Broadway Fl 2 (btwn 13th & 14th St), New ...","[{'label': 'display', 'lat': 40.73431360000001...",40.734314,-73.991184,NaN,10003,NY,45312392f964a5206e3b1fe3
8,YogaWorks Broadway and Grand,Yoga Studio,474 Broadway,US,New York,United States,at Grand,1004,"[474 Broadway (at Grand), New York, NY 10013, ...","[{'label': 'display', 'lat': 40.720861, 'lng':...",40.720861,-74.000868,NaN,10013,NY,4a3449eaf964a5200a9c1fe3
9,Yoga (JM),Yoga Studio,261 Broadway,US,New York,United States,Warren St,114,"[261 Broadway (Warren St), New York, NY 10007,...","[{'label': 'display', 'lat': 40.713753, 'lng':...",40.713753,-74.006027,NaN,10007,NY,58f8b7121c675b630de0b81a


In [115]:
dataframe_filtered.name.head()

0     Kula Yoga Project
1         Modo Yoga NYC
2             Yoga Vida
3    Yoga to the People
4             Yoga Vida
Name: name, dtype: object

In [116]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# add the Yoga restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [119]:
neighborhoods.loc[0, 'Neighborhood']

'Wakefield'

In [121]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Wakefield are 40.89470517661, -73.84720052054902.


In [123]:
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

In [125]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [127]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

10 venues were returned by Foursquare.


In [128]:
nearby_venues.head(10)

,name,categories,lat,lng
0,Lollipops Gelato,Dessert Shop,40.894123,-73.845892
1,Rite Aid,Pharmacy,40.896649,-73.844846
2,Carvel Ice Cream,Ice Cream Shop,40.890487,-73.848568
3,Cooler Runnings Jamaican Restaurant Inc,Caribbean Restaurant,40.898276,-73.850381
4,Shell,Gas Station,40.894187,-73.845862
5,SUBWAY,Sandwich Place,40.890656,-73.849192
6,Dunkin',Donut Shop,40.890459,-73.849089
7,Central Deli,Deli / Bodega,40.896846,-73.844415
8,Baychester Avenue Food Truck,Food Truck,40.892293,-73.843230
9,Koss Quick Wash,Laundromat,40.891281,-73.849904


In [129]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [131]:
NY_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )


Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [132]:
print(NY_venues.shape)
NY_venues.head()

(10386, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
2,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
3,Wakefield,40.894705,-73.847201,Cooler Runnings Jamaican Restaurant Inc,40.898276,-73.850381,Caribbean Restaurant
4,Wakefield,40.894705,-73.847201,Shell,40.894187,-73.845862,Gas Station


In [133]:
NY_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,27,27,27,27,27,27
Annadale,13,13,13,13,13,13
Arden Heights,4,4,4,4,4,4
Arlington,6,6,6,6,6,6
Arrochar,17,17,17,17,17,17
Arverne,19,19,19,19,19,19
Astoria,100,100,100,100,100,100
Astoria Heights,15,15,15,15,15,15
Auburndale,20,20,20,20,20,20


In [134]:
print('There are {} uniques categories.'.format(len(NY_venues['Venue Category'].unique())))

There are 424 uniques categories.


In [135]:
# one hot encoding
NY_onehot = pd.get_dummies(NY_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NY_onehot['Neighborhood'] = NY_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [NY_onehot.columns[-1]] + list(NY_onehot.columns[:-1])
NY_onehot = NY_onehot[fixed_columns]

NY_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repa

In [136]:
NY_onehot.shape

(10386, 424)

In [154]:
NY_grouped = NY_onehot.groupby('Neighborhood').mean().reset_index()


In [155]:
NY_grouped_Yoga = NY_grouped.sort_values('Yoga Studio',ascending=False)

NY_grouped_Yoga.head(10)

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bath House,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Bookstore,College Cafeteria,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repa

In [168]:
neighborhoods.loc[197, 'Neighborhood']

'St. George'

In [169]:
# one hot encoding
NY_onehot2 = pd.get_dummies(neighborhoods[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NY_onehot2['Neighborhood'] = neighborhoods['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns2 = [NY_onehot2.columns[-1]] + list(NY_onehot2.columns[:-1])
NY_onehot2 = NY_onehot2[fixed_columns2]

NY_onehot2.head()

KeyError: "None of [Index(['Venue Category'], dtype='object')] are in the [columns]"

In [167]:
NY_grouped_latitude = NY_grouped.loc[197, 'Latitude'] # neighborhood latitude value
NY_grouped_longitude = NY_grouped.loc[197, 'Longitude'] # neighborhood longitude value

NY_grouped_name = NY_grouped.loc[197, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

KeyError: 'Latitude'

In [138]:
NY_grouped.shape

(300, 424)

In [141]:
num_top_venues = 5

for hood in NY_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = NY_grouped[NY_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allerton----
           venue  freq
0    Pizza Place  0.15
1  Deli / Bodega  0.11
2    Supermarket  0.11
3            Spa  0.07
4    Gas Station  0.04


----Annadale----
                 venue  freq
0          Pizza Place  0.15
1  American Restaurant  0.15
2           Restaurant  0.08
3           Bagel Shop  0.08
4             Pharmacy  0.08


----Arden Heights----
         venue  freq
0     Pharmacy  0.25
1  Pizza Place  0.25
2  Coffee Shop  0.25
3     Bus Stop  0.25
4  Yoga Studio  0.00


----Arlington----
                 venue  freq
0         Intersection  0.17
1  American Restaurant  0.17
2         Liquor Store  0.17
3        Boat or Ferry  0.17
4             Bus Stop  0.17


----Arrochar----
                venue  freq
0       Deli / Bodega  0.12
1            Bus Stop  0.12
2  Italian Restaurant  0.12
3      Sandwich Place  0.06
4         Supermarket  0.06


----Arverne----
            venue  freq
0       Surf Spot  0.21
1  Sandwich Place  0.11
2   Metro Station  0.11
3     C

In [142]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [144]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = NY_grouped['Neighborhood']

for ind in np.arange(NY_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NY_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allerton,Pizza Place,Supermarket,Deli / Bodega,Spa,Martial Arts Dojo,Chinese Restaurant,Bakery,Fast Food Restaurant,Pharmacy,Gas Station
1,Annadale,Pizza Place,American Restaurant,Food,Dance Studio,Train Station,Sports Bar,Restaurant,Bagel Shop,Pharmacy,Diner
2,Arden Heights,Pharmacy,Coffee Shop,Bus Stop,Pizza Place,Fast Food Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor
3,Arlington,Liquor Store,Intersection,Food,American Restaurant,Boat or Ferry,Bus Stop,Women's Store,Field,Fast Food Restaurant,Farmers Market
4,Arrochar,Deli / Bodega,Italian Restaurant,Bus Stop,Mediterranean Restaurant,Middle Eastern Restaurant,Sandwich Place,Pizza Place,Athletics & Sports,Bagel Shop,Hotel


# Toronto Data

## Import data from Wikipedia

In [37]:
postalcodes_df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]

In [38]:
postalcodes_df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [40]:
postalcodes_df=postalcodes_df[postalcodes_df.Borough != 'Not assigned']

postalcodes_df['Neighbourhood']=postalcodes_df['Neighbourhood'].replace('Not assigned', postalcodes_df['Borough'])

postalcodes_df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [41]:
pc2_df=postalcodes_df.groupby(['Postcode','Borough'],as_index=False, sort=False).agg(','.join)
pc2_df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


In [42]:
print('The dataframe has {} boroughs and {} neighbourhoods.'.format(
        len(pc2_df['Borough'].unique()),
        pc2_df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighbourhoods.


In [43]:
coords_df = pd.read_csv("http://cocl.us/Geospatial_data")

coords_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
pc2_df.rename(columns={'Postcode': 'Postal Code'}, inplace=True)
pc2_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [45]:
df_final = pd.merge(pc2_df, coords_df, on='Postal Code')

df_final.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [47]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ca_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


In [56]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=FC5WMF5KAA2YQG5ICBTLLB1IL3POC450QMGGHLNYZMKSUAO5&client_secret=I4XMIHNH0OVB5MVYBAO4BPVRRRD4DWJQKUTUBNXWOD4GUQAI&ll=43.653963,-79.387207&v=20191008&query=Yoga&radius=20000&limit=200'

In [57]:
results = requests.get(url).json()

In [58]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

In [59]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,The Yoga Sanctuary,Yoga Studio,"2 College St. W, Suite 306",CA,Toronto,Canada,at Yonge St.,886,"[2 College St. W, Suite 306 (at Yonge St.), To...","[{'label': 'display', 'lat': 43.66149949759759...",43.661499,-79.383636,NaN,M5G 1K3,ON,4aeb5989f964a5203ec121e3
1,Yoga Tree Downtown,Yoga Studio,140 Spadina Ave Unit A201,CA,Toronto,Canada,at Richmond St,983,"[140 Spadina Ave Unit A201 (at Richmond St), T...","[{'label': 'display', 'lat': 43.64790246011444...",43.647902,-79.396086,NaN,M5V 2L4,ON,4d90ca3cc1b1721e93666346
2,Sivananda Yoga Centre,Yoga Studio,77 Harbord St,CA,Toronto,Canada,at Spadina Ave,1601,"[77 Harbord St (at Spadina Ave), Toronto ON M5...","[{'label': 'display', 'lat': 43.6627536, 'lng'...",43.662754,-79.402951,NaN,M5S 1G4,ON,4ba17867f964a520c7b937e3
3,Core Studio Yoga & Pilates,Yoga Studio,553 Queen St. W,CA,Toronto,Canada,at Augusta Ave.,1243,"[553 Queen St. W (at Augusta Ave.), Toronto ON...","[{'label': 'display', 'lat': 43.64791967343419...",43.647920,-79.400196,NaN,M5V 2B6,ON,4adf8284f964a520477b21e3
4,Tula Yoga Studio,Yoga Studio,"352 Adelaide St W., 3rd Floor",CA,Toronto,Canada,NaN,882,"[352 Adelaide St W., 3rd Floor, Toronto ON M5V...","[{'label': 'display', 'lat': 43.64746372311456...",43.647464,-79.393472,NaN,M5V 1R8,ON,53dea941498efed38c292d40
5,Aum Centre Yoga,Yoga Studio,NaN,CA,Toronto,Canada,NaN,1089,"[Toronto ON, Canada]","[{'label': 'display', 'lat': 43.657688, 'lng':...",43.657688,-79.399723,NaN,NaN,ON,5081d1e4e4b0921f5bb47c6f
6,Y Yoga,Yoga Studio,204 - 350 Queens Quay West,CA,Toronto,Canada,Between Spadina and Rees St.,1724,[204 - 350 Queens Quay West (Between Spadina a...,"[{'label': 'display', 'lat': 43.63853361362020...",43.638534,-79.389158,NaN,M5V 3A7,ON,50bf6e53d86ce1965377956b
7,Yoga Tree Bay and Dundas,Yoga Studio,123 Dundas Street West,CA,Toronto,Canada,NaN,282,"[123 Dundas Street West, Toronto ON, Canada]","[{'label': 'display', 'lat': 43.65556089904130...",43.655561,-79.384478,NaN,NaN,ON,542044a3498e78ca08c471bd
8,889 Yoga,Yoga Studio,550 Wellington Street West,CA,Toronto,Canada,Bathurst,1701,"[550 Wellington Street West (Bathurst), Toront...","[{'label': 'display', 'lat': 43.64283283704247...",43.642833,-79.401683,NaN,M5V 2V4,ON,4eb340f1e5e8743704d75e4a
9,Oxygen Yoga & Fitness,Yoga Studio,NaN,CA,Toronto,Canada,NaN,3039,"[Toronto ON M6K 3P6, Canada]","[{'label': 'display', 'lat': 43.638176, 'lng':...",43.638176,-79.417990,Liberty Village,M6K 3P6,ON,5d78433ea18949000809ebcb


In [60]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [74]:
venue_id = '4aeb5989f964a5203ec121e3' # ID of The Yoga Sanctuary
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4aeb5989f964a5203ec121e3?client_id=FC5WMF5KAA2YQG5ICBTLLB1IL3POC450QMGGHLNYZMKSUAO5&client_secret=I4XMIHNH0OVB5MVYBAO4BPVRRRD4DWJQKUTUBNXWOD4GUQAI&v=20191008'

In [76]:
result = requests.get(url).json()


In [77]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.3


In [78]:
venue_id = '4d90ca3cc1b1721e93666346' # ID of Yoga Tree Downtown
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.9
